In [71]:
import h5py
import os

def rename_h5_key(h5_path: str, old_key: str, new_key: str) -> None:
    """
    Rename a key in an HDF5 file and save the changes.
    
    Parameters:
        h5_path (str): Path to the HDF5 file.
        old_key (str): The existing key to rename.
        new_key (str): The new key name.
    """
    with h5py.File(h5_path, 'r+') as f:
        # Check if the old key exists
        if old_key in f:
            # Copy the data to the new key
            f[new_key] = f[old_key][:]
            
            # Copy attributes if any
            for attr_name, attr_value in f[old_key].attrs.items():
                f[new_key].attrs[attr_name] = attr_value
            
            # Delete the old key
            del f[old_key]
            print(f"Renamed '{old_key}' to '{new_key}' in file: {h5_path}")
        else:
            print(f"Key '{old_key}' not found in file: {h5_path}")
data_dir = '/ruiyan/yuhao/tile_embed/BCNB/TITAN'

# fets -> features
for file in os.listdir(data_dir):
    if file.endswith('.h5'):
        file_path = os.path.join(data_dir, file)
        rename_h5_key(file_path, old_key='feats', new_key='features')

Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_894.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_940.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_476.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_164.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_697.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_377.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_148.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_973.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_153.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_351.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_854.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_embed/BCNB/TITAN/bcnb_265.h5
Key 'feats' not found in file: /ruiyan/yuhao/tile_em

In [7]:
import pandas as pd
data = pd.read_csv('/ruiyan/yuhao/project/FMBC/finetune/dataset_csv/subtype/CPTAC_IDC.csv')
#map label columnt 0,1 to IDC and non-IDC
data['label'] = data['label'].map({1: 'IDC', 0: 'non-IDC'})
data.to_csv('/ruiyan/yuhao/project/FMBC/finetune/dataset_csv/subtype/CPTAC_IDC.csv', index=False)

In [22]:
#/ruiyan/yuhao/embedding/IMPRESS
import pandas as pd
import pandas as pd
df=pd.read_csv('/ruiyan/yuhao/project/FMBC/finetune/dataset_csv/sample/IMPRESS_TNBC_2subtype.csv')
# Assume `df` is your original dataframe
# Create a copy of the dataframe for '_he' and '_ihc'
df_he = df.copy()
df_ihc = df.copy()
#zfill slide_id column to 3 digits
df_he['slide_id'] =df['slide_id'].astype(str)+'_HE'
df_ihc['slide_id'] =df['slide_id'].astype(str)+'_IHC'



# Concatenate the two dataframes vertically
df_combined = pd.concat([df_he, df_ihc], ignore_index=True)

df_combined.to_csv('/ruiyan/yuhao/project/FMBC/finetune/dataset_csv/biomarker/IMPRESS_TNBC_2subtype.csv', index=False)

In [54]:
import os
import glob
data_dir= '/ruiyan/yuhao/tile_embed/TCGA-BRCA'
h5_files = glob.glob(os.path.join(data_dir, '**/*.h5'))

In [78]:
import os
from pathlib import Path
import glob
data_dir = '/ruiyan/yuhao/data'
num_patch=0
num_slides=0
for dir in os.listdir(data_dir):
    target_dir = os.path.join(data_dir, dir,'output')
        #find how many png file in the target_dir, glob
    png_files = glob.glob(os.path.join(target_dir, '**/*.png'))
    slides= len(os.listdir(os.path.join(data_dir, dir,'output')))
    num_slides+=slides
    #print(len(png_files))
    print(slides)
    print(len(png_files))
    num_patch+=len(png_files)

2788
8242984
2788
8138453
2696
8382726
2596
7728870


KeyboardInterrupt: 

In [2]:
data_dir='/ruiyan/yuhao/tile_embed/embedding/IMPRESS'
import os
for dir in os.listdir(data_dir):
    #Gigapath_tile -> Gigapath_Tile
    #CHIEF_tile -> CHIEF_Tile
    print(dir)
    if dir in ['Gigapath_tile', 'CHIEF_tile']:
        new_dir = dir.replace('_tile', '_Tile')
        os.rename(os.path.join(data_dir, dir), os.path.join(data_dir, new_dir))

Gigapath_tile
TITAN
CONCH
Gigapath
Virchow
UNI
CHIEF_tile


In [8]:
assets['features'].shape

(207, 768)

In [4]:
import pandas as pd

# Define the input and output file names
input_file = "KM_Plot__Overall_(months).txt"
output_file = "KM_Plot_Overall_(months).csv"

# Read the text file while handling whitespace and column names
df = pd.read_csv(input_file, delim_whitespace=True)

# Save the DataFrame to a CSV file
df.to_csv(output_file, index=False)

print(f"CSV file has been saved as {output_file}")


CSV file has been saved as KM_Plot_Overall_(months).csv


/tmp/ipykernel_117338/1885870243.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(input_file, delim_whitespace=True)


In [1]:
import pandas as pd

# 读取上传的数据文件
file_path = "/ruiyan/yuhao/project/FMBC/finetune/script/KM_Plot_Overall_(months).csv"
df = pd.read_csv(file_path)

# 查看数据的基本信息
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101 entries, 0 to 1100
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Study_ID    1101 non-null   object 
 1   Patient_ID  1101 non-null   object 
 2   OS_STATUS   1097 non-null   object 
 3   OS_MONTHS   1096 non-null   float64
dtypes: float64(1), object(3)
memory usage: 34.5+ KB


(None,
     Study_ID    Patient_ID OS_STATUS  OS_MONTHS
 0  brca_tcga  TCGA-3C-AAAU  0:LIVING     132.95
 1  brca_tcga  TCGA-3C-AALI  0:LIVING     131.57
 2  brca_tcga  TCGA-3C-AALJ  0:LIVING      48.42
 3  brca_tcga  TCGA-3C-AALK  0:LIVING      47.57
 4  brca_tcga  TCGA-4H-AAAK  0:LIVING      11.43)

In [2]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# 删除缺失值
df = df.dropna().reset_index(drop=True)

# 处理生存状态 (0:LIVING -> 0, 1:DECEASED -> 1)
df['OS_STATUS'] = df['OS_STATUS'].apply(lambda x: 1 if 'DECEASED' in x else 0)

# 对 Patient_ID 进行 Label Encoding（数值化）
patient_encoder = LabelEncoder()
df['Patient_ID'] = patient_encoder.fit_transform(df['Patient_ID'])

# 对 Study_ID 进行 Label Encoding（数值化）
study_encoder = LabelEncoder()
df['Study_ID'] = study_encoder.fit_transform(df['Study_ID'])

# 归一化生存时间
scaler = StandardScaler()
df['OS_MONTHS'] = scaler.fit_transform(df[['OS_MONTHS']])

# 划分特征和标签
X = df[['Patient_ID', 'Study_ID']].values  # 仅使用数值化的 Patient_ID 和 Study_ID 作为输入
y_time = df['OS_MONTHS'].values  # 归一化后的生存时间
y_status = df['OS_STATUS'].values  # 生存状态（0/1）

# 划分训练集和测试集
from sklearn.model_selection import train_test_split

X_train, X_test, y_time_train, y_time_test, y_status_train, y_status_test = train_test_split(
    X, y_time, y_status, test_size=0.2, random_state=42
)

# 查看数据处理后的结果
df.head()


,Study_ID,Patient_ID,OS_STATUS,OS_MONTHS
0,0,0,0,2.349389
1,0,1,0,2.314150
2,0,2,0,0.190885
3,0,3,0,0.169179
4,0,4,0,-0.753668


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
from lifelines import KaplanMeierFitter
from tqdm import tqdm

# 定义 NLLSurvLoss 损失函数
class NLLSurvLoss(nn.Module):
    def __init__(self, alpha=0.15):
        super(NLLSurvLoss, self).__init__()
        self.alpha = alpha
    
    def forward(self, hazards, S, Y, c, eps=1e-7):
        batch_size = len(Y)
        Y = Y.view(batch_size, 1) 
        c = c.view(batch_size, 1).float()
        if S is None:
            S = torch.cumprod(1 - hazards, dim=1)
        S_padded = torch.cat([torch.ones_like(c), S], 1)
        uncensored_loss = -(1 - c) * (torch.log(torch.gather(S_padded, 1, Y).clamp(min=eps)) + torch.log(torch.gather(hazards, 1, Y).clamp(min=eps)))
        censored_loss = - c * torch.log(torch.gather(S_padded, 1, Y+1).clamp(min=eps))
        neg_l = censored_loss + uncensored_loss
        loss = (1-self.alpha) * neg_l + self.alpha * uncensored_loss
        return loss.mean()

# 转换数据为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_time_train_tensor = torch.tensor(y_time_train, dtype=torch.float32)
y_status_train_tensor = torch.tensor(y_status_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_time_test_tensor = torch.tensor(y_time_test, dtype=torch.float32)
y_status_test_tensor = torch.tensor(y_status_test, dtype=torch.float32)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_time_train_tensor, y_status_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 定义 MLP 生存模型
class MLPSurvival(nn.Module):
    def __init__(self, input_dim):
        super(MLPSurvival, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 实例化模型
input_dim = X_train.shape[1]
model = MLPSurvival(input_dim)
loss_function = NLLSurvLoss()

# 训练模型
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 100
loss_history = []

for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0.0
    with tqdm(train_loader, unit="batch") as tepoch:
        for batch_X, batch_time, batch_status in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}/{n_epochs}")
            optimizer.zero_grad()
            preds = model(batch_X)
            loss = loss_function(preds, None, batch_time, batch_status)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            tepoch.set_postfix(loss=loss.item())
    
    loss_history.append(epoch_loss / len(train_loader))
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}")

# 绘制损失曲线
plt.plot(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.show()

# 计算生存曲线
model.eval()
preds_test = model(X_test_tensor).detach().numpy().flatten()
kmf = KaplanMeierFitter()
kmf.fit(y_time_test, event_observed=y_status_test)

plt.figure()
kmf.plot_survival_function()
plt.title("Kaplan-Meier Survival Curve")
plt.xlabel("Time (normalized)")
plt.ylabel("Survival Probability")
plt.show()


Epoch 1/100:   0%|          | 0/28 [00:00<?, ?batch/s]


RuntimeError: gather(): Expected dtype int64 for index